In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.style.use("seaborn-v0_8")


# Control cinemático del manipulador RR planar

A partir de la misma estructura generamos el análisis y el control para un manipulador de dos grados de libertad con ambas articulaciones rotacionales.

La cinemática directa y el Jacobiano se expresan en términos de las longitudes de los eslabones L1 y L2.

## 1. Parámetros

In [ ]:
params_rr = {
    'L1': 1.5,
    'L2': 1.0
}

sim_config_rr = {
    'kp': np.diag([2.0, 2.0]),
    't0': 0.0,
    'tf': 10.0,
    'n_steps': 600
}

t_rr, dt_rr = time_vector(sim_config_rr)


## 2. Cinemática directa

In [ ]:
def fkine_rr(q, params):
    q1, q2 = q
    L1, L2 = params['L1'], params['L2']
    x = L1 * np.cos(q1) + L2 * np.cos(q1 + q2)
    y = L1 * np.sin(q1) + L2 * np.sin(q1 + q2)
    return np.array([x, y])

fkine_rr(np.array([np.pi/6, -np.pi/3]), params_rr)


## 3. Jacobiano

In [ ]:
def jacobian_rr(q, params):
    q1, q2 = q
    L1, L2 = params['L1'], params['L2']
    s1, c1 = np.sin(q1), np.cos(q1)
    s12, c12 = np.sin(q1 + q2), np.cos(q1 + q2)
    return np.array([
        [-L1 * s1 - L2 * s12, -L2 * s12],
        [ L1 * c1 + L2 * c12,  L2 * c12]
    ])

jacobian_rr(np.array([0.0, 0.0]), params_rr)


### Derivación simbólica

In [ ]:
q1_rr, q2_rr, L1_rr, L2_rr = sp.symbols('q1 q2 L1 L2')
x_rr = L1_rr * sp.cos(q1_rr) + L2_rr * sp.cos(q1_rr + q2_rr)
y_rr = L1_rr * sp.sin(q1_rr) + L2_rr * sp.sin(q1_rr + q2_rr)
f_rr = sp.Matrix([x_rr, y_rr])
J_rr = f_rr.jacobian([q1_rr, q2_rr])
sp.simplify(J_rr)


## 4. Trayectoria

In [ ]:
Xd_rr, Vd_rr = circular_task_reference(radius=0.8, center=(1.2, 1.2), omega=2*np.pi, t=t_rr)
Xd_rr[:, :3]


## 5. Control

In [ ]:
def kinematic_control_step_rr(q, params, xd, xdp, kp):
    x = fkine_rr(q, params)
    J = jacobian_rr(q, params)
    omega = np.linalg.solve(J, xdp + kp @ (xd - x))
    return omega, x

kinematic_control_step_rr(np.array([0.0, 0.0]), params_rr, Xd_rr[:, 0], Vd_rr[:, 0], sim_config_rr['kp'])


## 6. Simulación

In [ ]:
def simulate_rr_control(q0, params, Xd, Vd, t, kp):
    n = t.size
    dt = t[1] - t[0]
    q = np.zeros((2, n))
    X = np.zeros((2, n))
    q[:, 0] = q0
    X[:, 0] = fkine_rr(q0, params)
    for k in range(1, n):
        omega, _ = kinematic_control_step_rr(q[:, k-1], params, Xd[:, k-1], Vd[:, k-1], kp)
        q[:, k] = q[:, k-1] + dt * omega
        X[:, k] = fkine_rr(q[:, k], params)
    return q, X

q0_rr = np.array([0.0, 0.0])
theta_rr, X_rr = simulate_rr_control(q0_rr, params_rr, Xd_rr, Vd_rr, t_rr, sim_config_rr['kp'])
X_rr[:, :3]


## 7. Visualización

In [ ]:
plot_tracking(X_rr, Xd_rr, t_rr)


### Animación

In [ ]:
def animate_rr(theta, X, Xd, t, params, axis_len=0.4, trail=True, skip=2):
    L1, L2 = params['L1'], params['L2']
    fig, ax = plt.subplots()
    ax.set_aspect('equal', adjustable='box')
    margin = 0.4
    x_all = np.concatenate([X[0], Xd[0], np.array([0.0])])
    y_all = np.concatenate([X[1], Xd[1], np.array([0.0])])
    ax.set_xlim(np.min(x_all) - margin, np.max(x_all) + margin)
    ax.set_ylim(np.min(y_all) - margin, np.max(y_all) + margin)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title('Robot RR planar')

    desired_line, = ax.plot(Xd[0], Xd[1], '--', linewidth=1.2, label='Trayectoria deseada')
    trail_line,   = ax.plot([], [], ':', linewidth=1.5, label='Rastro')
    link1_line, = ax.plot([], [], 'k-', linewidth=3)
    link2_line, = ax.plot([], [], 'k-', linewidth=3)
    ee_point,   = ax.plot([], [], 'o', markersize=5)

    f0_x, = ax.plot([], [], color='r', linewidth=2)
    f0_y, = ax.plot([], [], color='g', linewidth=2)
    f1_x, = ax.plot([], [], color='r', linewidth=2)
    f1_y, = ax.plot([], [], color='g', linewidth=2)
    f2_x, = ax.plot([], [], color='r', linewidth=2)
    f2_y, = ax.plot([], [], color='g', linewidth=2)
    base_dot,  = ax.plot([], [], 'o', color='b')
    joint_dot, = ax.plot([], [], 'o', color='b')
    ee_dot,    = ax.plot([], [], 'o', color='b')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    ax.legend(loc='upper right')

    def frame_axes(origin, angle):
        c, s = np.cos(angle), np.sin(angle)
        p = np.array(origin)
        return p, p + axis_len * np.array([c, s]), p + axis_len * np.array([-s, c])

    def init():
        trail_line.set_data([], [])
        for ln in (link1_line, link2_line, ee_point,
                   f0_x, f0_y, f1_x, f1_y, f2_x, f2_y):
            ln.set_data([], [])
        for dot in (base_dot, joint_dot, ee_dot):
            dot.set_data([], [])
        time_text.set_text('')
        return (
            link1_line, link2_line, ee_point, trail_line,
            f0_x, f0_y, f1_x, f1_y, f2_x, f2_y,
            base_dot, joint_dot, ee_dot, time_text
        )

    def update(frame):
        idx = min(frame * skip, t.size - 1)
        q1, q2 = theta[:, idx]
        O0 = np.array([0.0, 0.0])
        O1 = np.array([L1 * np.cos(q1), L1 * np.sin(q1)])
        O2 = np.array([
            L1 * np.cos(q1) + L2 * np.cos(q1 + q2),
            L1 * np.sin(q1) + L2 * np.sin(q1 + q2)
        ])

        link1_line.set_data([O0[0], O1[0]], [O0[1], O1[1]])
        link2_line.set_data([O1[0], O2[0]], [O1[1], O2[1]])
        ee_point.set_data([O2[0]], [O2[1]])

        if trail:
            trail_line.set_data(X[0, :idx+1], X[1, :idx+1])

        p0, p0x, p0y = frame_axes(O0, 0.0)
        f0_x.set_data([p0[0], p0x[0]], [p0[1], p0x[1]])
        f0_y.set_data([p0[0], p0y[0]], [p0[1], p0y[1]])
        base_dot.set_data([O0[0]], [O0[1]])

        p1, p1x, p1y = frame_axes(O1, q1)
        f1_x.set_data([p1[0], p1x[0]], [p1[1], p1x[1]])
        f1_y.set_data([p1[0], p1y[0]], [p1[1], p1y[1]])
        joint_dot.set_data([O1[0]], [O1[1]])

        p2, p2x, p2y = frame_axes(O2, q1 + q2)
        f2_x.set_data([p2[0], p2x[0]], [p2[1], p2x[1]])
        f2_y.set_data([p2[0], p2y[0]], [p2[1], p2y[1]])
        ee_dot.set_data([O2[0]], [O2[1]])

        time_text.set_text(f't = {t[idx]:.2f} s')
        return (
            link1_line, link2_line, ee_point, trail_line,
            f0_x, f0_y, f1_x, f1_y, f2_x, f2_y,
            base_dot, joint_dot, ee_dot, time_text
        )

    interval_ms = max(10, int((t[1] - t[0]) * 1000))
    frames = int(np.ceil(t.size / skip))
    anim = FuncAnimation(fig, update, frames=frames, init_func=init,
                         blit=True, interval=interval_ms)
    plt.close(fig)
    return anim

anim_rr = animate_rr(theta_rr, X_rr, Xd_rr, t_rr, params_rr, axis_len=0.4, trail=True, skip=2)
HTML(anim_rr.to_jshtml())
